# Fitting astronomical images and spectra

## What is a data cube?
A spectral line data cube is a type of astronomical data set that usual includes one spectral dimension and two spatial dimensions. The spectral dimension may represent the physical quantities of wavelength ($\lambda$), frequency ($\nu$), or velocity ($v$). Generally we call the images in the spectral dimension **channels**, which then have some width, $\Delta \lambda$ or $\Delta \nu$ or $\Delta v$. In the context of extragalactic astronomy, and usually *with respect to a particular spectral line*, these quantities are all connected by the definition of **redshift**, $z$:

$$ \frac{\Delta v}{c} = \frac{\Delta \lambda}{\lambda_{\rm obs}} = \frac{\Delta \nu}{\nu_{\rm obs}},$$

where $c$ is the speed of light, and for that spectral line, the observed wavelength is defined $\lambda_{\rm obs} = \lambda_{\rm rest} (1+z)$, and the observed frequency is defined $\nu_{\rm obs} = \nu_{\rm rest}/(1+z)$).

## What might a galaxy spectrum look like?
We often see a continuum component and spectral emission or absorption lines in a galaxy spectrum. This obviously will depend on the source galaxy, and at which wavelength/frequency/energy we're looking.

In our case, we will consider a power-law continuum of the form
$$ F_{\rm \nu}^{\rm (cont)} = F_{\nu, 0} \Big(\frac{\nu}{\nu_0}\Big)^{\alpha},$$
where $F_{\nu, 0}$ is the flux density at some point-of-reference frequency $\nu_0$. $\alpha$ is often called the **power-law slope** or **spectral index**, and this is because it is equal to the log-log slope of the flux density: $\alpha \equiv d \log F_\nu / d\log \nu$. Here, $\nu_0$ is arbitrary since it covaries with $F_{\nu, 0}$, and we can thus select a reference frequency without loss of generalization.

A simple spectral line model is a Gaussian distribution:
$$ F_{\rm \nu}^{\rm (line)} = \frac{F^{\rm (line)}}{\sqrt{2 \pi \sigma_\nu^2}}\exp \Bigg [ \frac{\big(\nu - \nu_{\rm rest}/(1+z)\big)^2}{2\sigma_\nu^2}\Bigg ]. $$
Here $\sigma_\nu$ parameterizes the width of the line (usually a function of velocity dispersion), and $F^{\rm (line)}$ is the total integrated line flux. $\nu_{\rm rest}/(1+z)$ is the observed (redshifted) spectral line frequency.

In total, we have five degrees of freedom: $F_\nu^{\rm (cont)}$, $\alpha$, $F^{\rm (line)}$, $z$, and $\sigma_\nu$. For example, suppose that we have a constant continuum (e.g., no frequency dependence so that $\alpha = 0$). We can then parameterize the **continuum + line** spectrum like
$$F_{\nu} = A \exp \Bigg [ \frac{\big(\nu - \nu_{\rm obs}\big)^2}{2\sigma_{\nu}^2} \Bigg] + K,$$
where $A$ is the Gaussian amplitude and proportional to the line flux, $\nu_{\rm obs}$ is the observed line frequency, and $\sigma^2$ is the Gaussian variance.


In [ ]:
# load pre-configured data
import numpy as np
import os
root_dir = os.path.join(os.getcwd(), '..', '..')
frequencies_fname = os.path.join(root_dir, 'data', 'frequencies.npy')
noiseless_cube_fname = os.path.join(root_dir, 'data', 'noiseless_cube.npy')

In [ ]:
# what are the dimensions of these arrays
frequencies = np.load(frequencies_fname)
print(frequencies.shape)

cube = np.load(noiseless_cube_fname)
print(cube.shape)

### A note on Numpy ordering
Here, the dimensions `(256, 32, 32)` signify that the zeroth axis has length `256`, and the first and second axes have length `32`. I will use the indicies `k, i, j` to describe these axes respectively.

The length of the zeroth axis is actually the *third* axis in Fortran notation (I'm sorry if this is getting confusing)! All that really matters is that `256` is the length of the **spectral** axis, and that `32` $\times$ `32` is the shape of each **image** or **channel**.

If we want to visualize the spectrum, we will need to figure out how to compress the `32` $\times$ `32` image into a single number at every frequency. This can be done by taking the mean, sum, median, selcting a particular pixel, etc.

In [ ]:
N_k, N_i, N_j = cube.shape

# let's visualize the spectrum through pixel 16, 16
spectrum = cube[:, 16, 16]

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10, 4))
plt.plot(frequencies, spectrum)
plt.ylim(0, 8)

plt.xlabel('Frequencies [GHz]')
plt.ylabel('Spectrum');

### Results
In this simple example, we can see that the spectrum has a constant flux density of $\sim 1.3$ in whatever units, and that htere is a line at a frequency slightly lower than 98 GHz. (I'm just saying that the units are in GHz but that isn't something you can tell from our data.)

Next, we will work on modeling the data to see if we can estimate the model parameters, which in turn should tell us something about the astrophysical phenomena responsible for the continuum and spectral line emission.

## Exercise 1
Write a function, `model_spectrum()`, that will output the flux density at multiple frequencies, `x` (equivalent to $\nu$ above, according to a given set of parameters: `line_amplitude`, `line_frequency`, `line_sigma`, and `continuum_amplitude`.

We will supply some parameters for the model (or mock) spectrum. Both the model and the data will be plotted in a cell below. Your answer should look something like this: ![Exercise 1 answer](../../doc/answers/1-1_ex1.png)

In [ ]:
def model_spectrum(x, *parameters):
    """Returns an array of the same length as `x` (given) that 
    produces a model of a spectrum with a flat continuum and
    single spectral emission line.
    """
    
    # unpack the parameters
    line_amplitude, line_frequency, line_sigma, continuum_amplitiude = parameters
    
    model = np.zeros_like(x)
    
    # fill out code below
    # -------------------    
    
    return model


In [ ]:
# test your code
initial_parameters = [6, 97.8, 0.1, 1.3]
mock_spectrum = model_spectrum(frequencies, *initial_parameters)

plt.figure(figsize=(12, 4))
plt.plot(frequencies, mock_spectrum, label='Mock spectrum')
plt.plot(frequencies, spectrum, label='Data')
plt.ylim(0, 8)
plt.xlabel('Frequencies [GHz]')
plt.ylabel('Flux density [arb.]')
plt.legend();

## Fitting a model to the spectrum

How would you fit your model to the data (spectrum) above?

When we talk of fitting models to data, we generally need to minimize some *objective* or *cost* function. A simple but good choice of cost function here is the **squared error**:
$$ J(\vec \theta) = \sum_\nu \big |\big | F_\nu - \hat y(\nu; \vec \theta)  \big | \big|^2, $$
where $F_\nu$ is the spectrum at every frequency $\nu$, and $\hat y$ is your model (a Gaussian spectral line plus flat continuum) is a function of frequency, $\nu$, and depends on **model parameters**, $\vec\theta$, from before: 
$$\vec \theta = \begin{pmatrix} A\\ \nu_{\rm obs}\\ \sigma_\nu\\ K\end{pmatrix}.$$

## Exercise 2
Implement the squared error cost function in the cell below.

In [ ]:
def squared_error(model_params, x, y):
    """Returns the sum of the squared error between the 
    data and a model constructed from the given model 
    parameters at every channel.
    """
    
    squared_error = 0
    
    # fill out code below
    # -------------------
        
    # hint: generate a model spectrum and then take the squared error
    
    return squared_error

We will evaluate the squared_error of our first model with guessed parameters. 

You should find that the summed square error is approximately equal to **70.5**.

In [ ]:
# evaluate squared error
J = squared_error(initial_parameters, frequencies, spectrum)
print('The squared error is {:.2f}'.format(J))

By the way, sometimes people use the mean squared error cost function, $$J(\vec\theta) = \frac{1}{N_k} \sum_\nu \big|\big| F_\nu - \vec y(\nu; \vec \theta)\big|\big|^2,$$
which only differs from the summed squared error by a factor of $N_k$, the length of the vector of frequencies. 

## Exercise 3
Inspect the `scipy.optimize.minimize` documentation. Note that you must supply an objective function (which in our case is `squared_error` --- make sure not to include the parentheses as that would signify a function *call*), an initial guess of parameters, and other optional arguments such as bounds, a matrix of derivatives, and detailed implementations of the minimizer. Don't worry about the options for now.

Execute the `scipy.optimize.minimize` command with appropriate arguments/parameters. The routine should *converge* on a global minimum. Save the output, including its final cost and **least squares** model parameter values.

You should find that the best-fit model parameters are about:

      line amp       =  6.0
      line freq      = 97.8
      line sigma     =  0.1
      continuum amp  =  1.3

In [ ]:
import scipy.optimize

# execute the minimization routine below and save result
# ------------------------------------------------------

optimize_result = scipy.optimize.minimize?

In [ ]:
if optimize_result['success']:
    print('😄')
    final_cost = optimize_result['fun']
    final_parameters = optimize_result['x']
    parameter_names = ['line amp', 'line freq', 'line sigma', 'continuum amp']
    
    print('The best-fit parameters are:')
    for param, value in zip(parameter_names, final_parameters):
        print('  {:<15s}= {:>4.1f}'.format(param, value))
else:
    print('😢')

## What does a channel look like?
If an astronomical source is *unresolved* -- that is, its angular size is smaller than the point-spread function (PSF) or synthesized radio beam, then its shape will be well approximated by either the PSF or the beam. In practice, these images usually take the shape of a $2d$ ellipse.

## Exercise 3
Fill out the code below in order to plot an image of the peak channel, and mark the location of the peak channel with an `x` marker. 
1. Find `k0`, the channel number where the spectrum is at a maximum. You can do so using the function `np.nanargmax`, or by finding the minimum value of $|\nu - \nu_{\rm obs}|$, where $\nu_{\rm obs}$ was found via least squares regression.
2. Save the image at channel `k0` in the data cube to the variable `image` and plot it.
3. Find `i0` and `j0`, the coordinates of the maximum pixel of `image`. This was accomplished in [notebook 0](0. Introduction to Python%2C Jupyter notebooks%2C and the Scipy stack.ipynb).

Your answer should look something like ![Exercise 3 answer](../../doc/answers/1-1_ex3.png "Peak pixel and channel in data cube")

In [ ]:
# fill out code below
# -------------------
k0 = 0
image = np.zeros((N_i, N_j))

i0, j0 = 0

# don't change anything below this line
# -------------------------------------

plt.imshow(image, origin='lower')
plt.colorbar()

# note that ij notation is column-major, not row-major like Numpy
plt.scatter(j0, i0, marker='x', color='C3') 

plt.ylabel('i')
plt.xlabel('j')

plt.show()

## A spatial grid

At this point, we are ready to make a $2d$ Gaussian model for the spatial component of the data cube, e.g., `image`. Just as we had previously used an array of frequencies over which we called the function `model_spectrum`, we will now need a $2d$ grid of positions, $i$ and $j$, which determine the pixel's value (flux density, or technically the *brightness distribution*).

The function that we will use is `np.meshgrid`, which converts the vectors
$$\vec i = \begin{pmatrix}1\\2\\ \vdots \\ N_i\end{pmatrix}, ~~~~~ \vec j = \begin{pmatrix}1\\2\\ \vdots \\ N_j\end{pmatrix},$$
into two matrices which respectively determine the $i$ and $j$ values across the $N_i \times N_j$ grid. This means that

$$ii = \begin{pmatrix}0&0&\cdots&0\\1&1&\cdots&1\\ \vdots&\vdots&\ddots&\vdots\\N_i&N_i&\cdots&N_i\end{pmatrix}, ~~~~~~~~~~
jj = \begin{pmatrix}0&1&\cdots&N_j\\0&1&\cdots&N_j\\ \vdots&\vdots&\ddots&\vdots\\0&1&\cdots&N_j\end{pmatrix}.$$

In [ ]:
ii, jj = np.meshgrid(range(N_i), range(N_j), indexing='ij')

### The spatial model
We can imagine a simple model to describe a single channel of the unresolved astronomical source using a $2d$ circular Gaussian. The formula required is:

$$ f(i,j) = A \exp \Bigg [ \frac{(i-i_0)^2 + (j-j_0)^2}{2\sigma_r^2} \Bigg],$$
where $A$ describes the amplitude (again), $i_0$ and $j_0$ are the $ij$-indexed center coordinates, and $\sigma_r$ parameterizes the circular Gaussian width. Note that the amplitude in the spatial model, $A$, is the same as the amplitude of the spectral model at the brightest location in space.

The brightness model parameters here are

$$ \vec \theta = \begin{pmatrix}A \\ i_0 \\ j_0 \\ \sigma_r\end{pmatrix}.$$ 

## Exercise 4
Now write a function that generates a $2d$ Gaussian model of the brightness distribution similar to the figure of `image` you plotted above. Remember that your model is a function of the "grid" variables `ii` and `jj` as well as the model parameters. I suggest that you write it in the form

    def model_brightness(grid, *theta):
        ii, jj = grid
        amp, i0, j0, sigma = theta
        
        ...

such that you can later call the function using `model_brightness((ii, jj), *theta)`. This will make it much easier for future implementation using `scipy.optimize.minimize`, which requires both `ii` and `jj` but should not vary either of them! (On the other hand, the model parameters, `theta`, *should* be varied in order to minimize the objective function.)

In [ ]:
def model_brightness(grid, *theta):
    """Given a tuple of (ii, jj) in addition to the model parameters
    theta, which consist of the amplitude, center i and j coordinates,
    and sigma, return a circularly symmetric 2-dimensional Gaussian.
    """
    ii, jj = grid
    amp, i0, j0, sigma = theta  
    
    brightness = np.zeros_like(ii)
    
    # complete code below
    # -------------------
    
    return brightness
    

### Checking our answers
Now we're going to plot a model with guessed parameters, the actual data, and a difference image between the two. It should look something like this:
![Exercise 4 answer](../../doc/answers/1-1_ex4.png)

In [ ]:
# feed guessed parameters into model
initial_spatial_params = [8., 14, 17, 3]
model = model_brightness((ii, jj), *initial_spatial_params)

image = cube[np.nanargmax(spectrum)]

difference = image - model

# plotting -- don't worry too much about what's going on here
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, [ax1, ax2, ax3] = plt.subplots(1, 3, sharey=True, figsize=(12, 4))

im1 = ax1.imshow(image, origin='lower', cmap='viridis')
ax1.text(0.05, 0.9, 'Data', size=14, color='white', transform=ax1.transAxes)
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes('right', size='5%', pad=0.04)
fig.colorbar(im1, cax=cax1, orientation='vertical')

im2 = ax2.imshow(model, origin='lower', cmap='viridis')
ax2.text(0.05, 0.9, 'Model', size=14, color='white', transform=ax2.transAxes)
divider = make_axes_locatable(ax2)
cax2 = divider.append_axes('right', size='5%', pad=0.04)
fig.colorbar(im2, cax=cax2, orientation='vertical')

im3 = ax3.imshow(difference, cmap='RdBu_r', vmin=-1, vmax=1, origin='lower')
ax3.text(0.05, 0.9, 'Data - model', size=14, color='black', transform=ax3.transAxes)
divider = make_axes_locatable(ax3)
cax3 = divider.append_axes('right', size='5%', pad=0.04)
fig.colorbar(im3, cax=cax3, orientation='vertical');

## Exercise 5

Now try minimizing the squared error cost function in order to conform your spatial model to the data. Note that you will need to redefine the `squared_error` function so that it evaluates the error between the spatial data and model (not the spectral data and spectral model from before).

You should find an answer like below (note that the difference image has colorbar ranging $\pm 5 \times 10^{-7}$!
![Exercise 5 solution](../../doc/answers/1-1_ex5.png)

In [ ]:
def squared_error(model_params, ii, jj, image):
    """Returns the sum of the squared error between the 
    spatial data and the model using the given brightness
    model parameters.
    """
    
    squared_error = 0
    
    # fill out code below
    # -------------------
        
    return squared_error

In [ ]:
image = cube[np.nanargmax(spectrum)]

# write code to find optimized model that describes `image`
# ---------------------------------------------------------



# plotting -- don't change code below
final_model = model_brightness((ii, jj), *final_spatial_parameters)
difference = image - final_model

fig, [ax1, ax2, ax3] = plt.subplots(1, 3, sharey=True, figsize=(12, 4))

im1 = ax1.imshow(image, origin='lower', cmap='viridis')
ax1.text(0.05, 0.9, 'Data', size=14, color='white', transform=ax1.transAxes)
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes('right', size='5%', pad=0.04)
fig.colorbar(im1, cax=cax1, orientation='vertical')

im2 = ax2.imshow(final_model, origin='lower', cmap='viridis')
ax2.text(0.05, 0.9, 'Optimized model', size=14, color='white', transform=ax2.transAxes)
divider = make_axes_locatable(ax2)
cax2 = divider.append_axes('right', size='5%', pad=0.04)
fig.colorbar(im2, cax=cax2, orientation='vertical')

im3 = ax3.imshow(difference, cmap='RdBu_r', vmin=-5e-7, vmax=5e-7, origin='lower')
ax3.text(0.05, 0.9, 'Data - model', size=14, color='black', transform=ax3.transAxes)
divider = make_axes_locatable(ax3)
cax3 = divider.append_axes('right', size='5%', pad=0.04)
fig.colorbar(im3, cax=cax3, orientation='vertical');